In [88]:
import pandas as pd
from sklearn import preprocessing

In [132]:
df = pd.read_csv("Data/ticket.csv", delimiter="\t")

#### Removing redundant sections and rows with NaN for CheckinTime

In [133]:
df.drop("TIME SECTION",1, inplace = True)
df.drop("TICKET",1, inplace = True)
df.drop("TICKET NUM",1, inplace = True)
df.drop("TABLE TYPE",1, inplace = True)
df.drop("STATUS",1, inplace = True)
df.drop("CHANNEL",1, inplace = True)
df = df.dropna()

In [136]:
df.head(5)

,REST_ID,TABLE SIZE,CREATE TIME,CHECKINTIME,TIME (HR),TIME (SEC)
0,113442,5,2016-01-01 11:09:19.000,2016-01-01 11:16:41,11,559
1,113442,4,2016-01-01 11:15:45.000,2016-01-01 11:16:40,11,945
2,113442,2,2016-01-01 11:17:11.000,2016-01-01 11:17:21,11,1031
3,113442,2,2016-01-01 11:34:41.000,2016-01-01 11:34:53,11,2081
4,113442,5,2016-01-01 11:51:00.000,2016-01-01 12:00:36,11,3060


In [135]:
df["TIME (HR)"] = df["CREATE TIME"].apply(lambda a: a.split()[1].split(':')[0])
df["TIME (SEC)"] = df["CREATE TIME"].apply(lambda a: int(int(a.split()[1].split(':')[1])*60 + float(a.split()[1].split(':')[2])))
df["MONTH"] = df["CREATE TIME"].apply(lambda a: int(a.split()[1].split('-')[1]))
df["TIME (HR)"] = df["CREATE TIME"].apply(lambda a: DateTime )

In [75]:
# lb = preprocessing.LabelBinarizer()
# lb.fit(['01','02','03','04','05','06','07','08','09','10','11','12'])
def handleDate(i, DT):
    date, time = DT.split()
    print(date)
    yr, mth, dd = date.split('-')
#     mth = lb.transform([mth])[0]
    mth = int(mth)
    hr, mn, sec = time.split(':')
    sec = int(int(mn)*6 + float(sec))
    hr = int(hr)
    test.loc[i, ["TIME HR"]] = hr
    test.loc[i, ["TIME SEC"]] = sec
    test.loc[i, ["MONTH"]] = mth
    

In [98]:
test.loc[:,["CREATE TIME"]].items()

<generator object DataFrame.iteritems at 0x0000021C8E2369E8>

In [109]:
i = 0
for j,k in test.loc[:,["CREATE TIME"]].items():
#     if i !=0:
#         handleDate(i,j)
#         pass
    i = i+1
    for yo,elem in enumerate(k):
        handleDate(yo, elem)
# map(handleDate, test.loc[1:,["CREATE TIME"]])

2016-01-01
2016-01-01
2016-01-01

C:\Users\user\Anaconda3\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel\__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel\__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr


2016-01-01
2016-01-01


In [110]:
test

,REST_ID,TABLE SIZE,CREATE TIME,CHECKINTIME,TIME HR,TIME SEC,DAY,MONTH,WAIT TIME
0,113442,5,2016-01-01 11:09:19.000,2016-01-01 11:16:41,11,73,,1,
1,113442,4,2016-01-01 11:15:45.000,2016-01-01 11:16:40,11,135,,1,
2,113442,2,2016-01-01 11:17:11.000,2016-01-01 11:17:21,11,113,,1,
3,113442,2,2016-01-01 11:34:41.000,2016-01-01 11:34:53,11,245,,1,
4,113442,5,2016-01-01 11:51:00.000,2016-01-01 12:00:36,11,306,,1,
